<h1 align="center">Deep Learning - Master in Deep Learning of UPM</h1> 

**IMPORTANTE**

Usaremos PyTorch Lightning para unificar arquitectura y lógica de entrenamiento en un único módulo:

In [35]:
# %%capture
# !pip install pytorch_lightning

En esta sesión implementaremos _from scratch_ un Transformer utilizando únicamente `PyTorch` para construir la arquitectura y `Lightning` para el entrenamiento.

In [140]:
import torch
import torch.nn as nn
import pytorch_lightning

# Tokenizador

## Character-level tokenizer

In [148]:
class CharTokenizer:
    def __init__(self):
        self.vocab_size = 0
        self.str2tok = {}
        self.tok2str = {}

    @classmethod
    def from_data(cls, data):
        tokenizer = cls()

        vocab = sorted(list(set(data)))
        tokenizer.vocab_size = len(vocab)

        tokenizer.str2tok = {c: t for t, c in enumerate(vocab)}
        tokenizer.tok2str = {t: c for t, c in enumerate(vocab)}

        return tokenizer

    def encode(self, text):
        _encode = lambda text: [self.str2tok[c] for c in text]

        if isinstance(text, list):
            return [_encode(s) for s in text]
        else:
            return _encode(text)
    
    def decode(self, tokens):
        return "".join([self.tok2str[t] for t in tokens])

# [EJERCICIO] TinyNextToken Model

## Carga del Dataset

En este bloque de código, cargamos el conjunto de datos que utilizaremos para entrenar. Utilizamos concretamente el texto de Shakespeare, pero podéis probar con otros textos si lo deseáis.

In [ ]:
data_path = 'data/shakespeare.txt'

with open(data_path, 'r') as f:
    data = f.read()

print(data[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


## Dataset

En autoregresión, el objetivo es predecir el siguiente token en una secuencia dada una serie de tokens anteriores. Por lo tanto, para cada secuencia de entrada, el objetivo (target) será la misma secuencia desplazada una posición hacia la derecha.

Para la secuencia de entrada:

```"The cat sat on the mat"```

La entrada correspondiente será:

```"The cat sat on the"```

El objetivo correspondiente será:

```"cat sat on the mat"```

De esta forma:
- "The" predice "cat"
- "The cat" predice "sat"
- "The cat sat" predice "on"
- Y así sucesivamente.

In [32]:
class DatasetForAutoregression(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, seq_len=32):
        self.data = data
        self.tokenizer = tokenizer
        self.seq_len = seq_len + 1 # +1 for target

    def __len__(self):
        return len(self.data) - self.seq_len + 1
    
    def __getitem__(self, idx):
        start_idx, end_idx = idx, idx+self.seq_len
        sequence = torch.tensor(self.tokenizer.encode(self.data[start_idx:end_idx]))
        inputs = sequence[:-1]
        targets = sequence[1:]
        return inputs, targets

## DataModule

In [33]:
class DataModuleForAutoregression(pytorch_lightning.LightningDataModule):
    def __init__(self, data, tokenizer, max_seq_len=32, batch_size=64, train_size=.8, multiprocess_config={}):
        super().__init__()
        self.data = data
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        self.batch_size = batch_size
        self.train_size = train_size

        self.multiprocess_config = multiprocess_config
        
    def setup(self, stage=None):
        if stage == 'fit':
            dataset = DatasetForAutoregression(self.data, self.tokenizer, seq_len=self.max_seq_len)

            train_size = int(len(dataset) * self.train_size)
            val_size = len(dataset) - train_size
            self.train_dataset, self.val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, **self.multiprocess_config)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset, batch_size=self.batch_size, **self.multiprocess_config)

## Arquitectura TinyNextToken

Vamos a implementar un pequeño modelo compuesto únicamente por una única capa de Embedding para predecir el siguiente token.

El truco aquí es usar un `nn.Embedding` con `vocab_size` tanto para la dimensión de entrada como para la de salida. De esta forma, cada vector de embedding puede ser interpretado como un *logit* para cada token en el vocabulario.

Tokens [batch_size, seq_len] -> Embedding [batch_size, seq_len, vocab_size] -> Predecimos el siguiente token.

La función de pérdida será una `CrossEntropyLoss` estándar, pero ¿cómo funciona la autorregresión en este caso?

1. Durante el entrenamiento, para cada secuencia de entrada, desplazamos los tokens de salida una posición a la izquierda. Esto significa que el modelo aprende a predecir el token siguiente en la secuencia.
    - Input: [The cat sat on the]
    - Target: [cat sat on the mat]

2. La función de pérdida compara las predicciones del modelo con los tokens objetivo desplazados, calculando la pérdida solo en las posiciones correspondientes a los tokens reales.

In [17]:
import torch
import torch.nn as nn
import pytorch_lightning as pl

class TinyNextTokenModule(pl.LightningModule):
    def __init__(
        self,
        vocab_size,
        lr=1e-3,
        max_seq_len=32,
        pad_token=0,
    ):
        super().__init__()
        self.save_hyperparameters()

        self.embedding = nn.Embedding(vocab_size, vocab_size) # Embedding layer
        self.criterion = nn.CrossEntropyLoss() # Classification loss for autoregression

        self.lr = lr # Learning rate
    
    def forward(self, input_ids):
        x = self.embedding(input_ids)
        return x

    def compute_loss(self, batch, split='train'):
        inputs, targets = batch
        logits = self(inputs)
        
        B, T, C = logits.size()
        logits = logits.reshape(B * T, C)
        targets = targets.reshape(B * T)

        loss = self.criterion(logits, targets)

        self.log(
            f'{split}_loss',
            loss,
            on_step=(split=='train'),
            on_epoch=True,
            prog_bar=(split=='train')
        )
        return loss
    
    def training_step(self, batch, batch_idx):
        return self.compute_loss(batch, split='train')
    
    def validation_step(self, batch, batch_idx):
        return self.compute_loss(batch, split='val')
    
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr)

In [141]:
# import os
# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'

In [61]:
# Hyperparameters
tokenizer = CharTokenizer.from_data(data)
max_seq_len = 128
batch_size = 32
train_size = 0.8
learning_rate = 1e-3
training_steps = 5_000

# Create DataModule
data_module = DataModuleForAutoregression(
    data=data,
    tokenizer=tokenizer,
    max_seq_len=max_seq_len,        # puedes cambiarlo
    batch_size=batch_size,          # bastante seguro
    train_size=train_size,
    multiprocess_config={'num_workers': 2}
)

module = TinyNextTokenModule(
    vocab_size=tokenizer.vocab_size,
    lr=learning_rate,
)

trainer = pl.Trainer(
    max_steps=training_steps,              
    accelerator="gpu",
    devices=[5],
)
    

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Definamos una función que nos permita generar texto. Funcionará de la siguiente manera:
- Dada una secuencia inicial (text), el modelo predice el siguiente token. El _cual_ elige se conoce como sampling y es algo que veremos más adelante.
- Ese token se añade a la secuencia.
- El proceso se repite hasta alcanzar la longitud deseada.

In [65]:
import torch
import torch.nn.functional as F

def generate(model, tokenizer, text, max_new_tokens=100, temperature=1.0, top_k=40):
    """
    Autoregressive text generation with top-k sampling and temperature.
    - model: trained TinyNextTokenModule (or compatible)
    - tokenizer: tokenizer with encode/decode methods
    - text: initial prompt string
    """
    model.eval()
    device = next(model.parameters()).device

    # Encode prompt → tensor shape: (1, seq_len)
    input_ids = torch.tensor([tokenizer.encode(text)], dtype=torch.long, device=device)

    for _ in range(max_new_tokens):
        # Forward pass → (1, seq_len, vocab_size)
        with torch.no_grad():
            logits = model(input_ids)

        # Select last timestep logits → (vocab_size,)
        logits = logits[0, -1]

        # Apply temperature
        logits = logits / temperature

        # Optional top-k filtering
        if top_k is not None:
            values, indices = torch.topk(logits, top_k)
            mask = torch.full_like(logits, float('-inf'))
            mask[indices] = logits[indices]
            logits = mask

        # Convert to probabilities
        probs = F.softmax(logits, dim=-1)

        # Sample from distribution
        next_token = torch.multinomial(probs, num_samples=1).item()

        # Append token → new shape (1, seq_len+1)
        next_token_tensor = torch.tensor([[next_token]], device=device)
        input_ids = torch.cat([input_ids, next_token_tensor], dim=1)

    # Decode entire sequence (move to CPU first)
    return tokenizer.decode(input_ids[0].cpu().tolist())


Vemos que previo a ser entrenado genera texto sin ningún sentido.

In [ ]:
print(generate(module, tokenizer, text="To be, or not to be, that is the", max_new_tokens=100))

"To be, or not to be, that is theKS\n.,bcnxVNWqV;F-&KI?;I'uMEgRHERNcR?!AroNyYc&dx'YxVg.a-$eDE,niE3NLdS\ndwhAUmHE.iNpk!PclE,tr.rQyRuArq\n"

Entrenemos esperando que al menos aprenda cierta _forma_ del corpus. (Puede llevar un par de minutos).

In [64]:
trainer.fit(module, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5]

  | Name      | Type             | Params | Mode
------------------------------------------------------
0 | embedding | Embedding        | 4.2 K  | eval
1 | criterion | CrossEntropyLoss | 0      | eval
------------------------------------------------------
4.2 K     Trainable params
0         Non-trainable params
4.2 K     Total params
0.017     Total estimated model params size (MB)
0         Modules in train mode
2         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/adrian/miniconda3/envs/synthetic-generation/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:527: Found 3 module(s) in eval mode at the start of training. This may lead to unexpected behavior during training. If this is intentional, you can ignore this warning.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=5000` reached.


In [68]:
print(generate(module, tokenizer, text="To be, or not to be, that is the", max_new_tokens=100))

To be, or not to be, that is the n h amajur nl! sls wous, tory ton?
Whass t:
S:
A llly won lewouth
T:
Bure pr nge m CENG ng pof po g


Este modelo es extremadamente simple y nos sirve como baseline para adentrarnos en el mundo de los Transformers. Aquí los tokens no están _hablando entre ellos_ ya que no hay intercambio de información entre posiciones. Realmente para predecir el siguiente token, el modelo solo puede basarse en la información del token actual, concretamente en el embedding para ese token específico, que además hace de logit. Esto quiere decir que cada embedding en este caso va a intentar maximizar la probabilidad del token que le sigue en el dataset.

El embedding correspondiente a cada token será una especie de media ponderada de los tokens que le siguen en el dataset. Esto por supuesto no es suficiente para generar texto coherente.